In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 61 kB 459 kB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
     |████████████████████████████████| 3.3 MB 32.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np

MODEL = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

roberta = AutoModel.from_pretrained(MODEL).to(device)

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
import pandas as pd
df = pd.read_csv("reddit-dataset.csv", sep="_____", error_bad_lines = False)
X = df["bel,comment"].tolist()
X = [preprocess(str(x)) for x in X]
y = list(df["l"])
X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
Skipping line 35668: Expected 2 fields in line 35668, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 43793: Expected 2 fields in line 43793, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 44948: Expected 2 fields in line 44948, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 47085: Expected 2 fields in line 47085, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 69173: Expected 2 fields in line 69173, saw 3. Error could possibly be due to qu

['NC and NH.',
 'You do know west teams play against west teams more than east teams right?',
 "They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",
 'This meme isn\'t funny none of the ""new york nigga"" ones are.',
 'I could use one of those tools.',
 "I don't pay attention to her, but as long as she's legal I wouldn't kick her out of bed (before she took a load)",
 'Trick or treating in general is just weird...',
 'Blade Masteryㆫ\u1a9a\ue9de or GTFO!',
 "You don't have to, you have a good build, buy games or save it",
 'I would love to see him at lolla.',
 'I think a significant amount would be against spending their tax dollars on other people.',
 'Damn I was hoping God was real',
 'They have an agenda.',
 'Great idea!',
 'Ayy bb wassup, it makes a bit more sense in context',
 'what the fuck',
 'noted.',
 "because it's what really bothers him... and it's a sign of weakness.",
 'why you fail me, my precious?',
 '

In [ ]:
from torch.nn.functional import pad
X_encoded  = [tokenizer(xx, return_tensors='pt')['input_ids'].long() for xx in X]

In [ ]:
X_train, X_cross, y_train, y_cross = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [ ]:
import torch
from torch import nn


class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = roberta
        self.linear1 = nn.Linear(4608, 800)
        self.linear2 = nn.Linear(800, 20)
        self.linear3 = nn.Linear(20, 2)
        self.pooling = nn.MaxPool2d((20, 1))
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = self.roberta(x).last_hidden_state
        x = self.pooling(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [ ]:
def trim_or_pad(tensor, n):
  length = tensor.shape[1]
  if length >= n :
    return torch.squeeze(tensor[:, :n])
  else :
    return torch.squeeze(torch.cat([tensor, torch.ones(1, n - length).long()], dim = 1))

In [ ]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __getitem__(self, index):
        x = trim_or_pad(self.data[index], 127)
        y = torch.squeeze(torch.Tensor([self.targets[index]])).type(torch.long)
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset_train = MyDataset(X_train, y_train) # create your datset
dataloader_train = DataLoader(dataset_train,batch_size=30) # create your dataloader
dataset_cross = MyDataset(X_cross, y_cross) # create your datset
dataloader_cross = DataLoader(dataset_cross,batch_size=30) # create your dataloader

In [ ]:
def train(dataloader, model, loss_fn, optimizer): #fragment z dokumentacji pytorch
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss :>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= num_batches
    print(f"Avg train loss: {train_loss:>8f} \n")


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    true_positive, true_negative, false_negative, false_positive = 0.0001, 0.0001, 0.0001, 0.0001 #zeby dzielenia przez zero nie bylo
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            y_pred = pred.argmax(1)
            correct += (y_pred == y).type(torch.long).sum().item()

            true_positive += torch.logical_and(y_pred == y, y == 1).sum().item()
            true_negative += torch.logical_and(y_pred == y, y == 0).sum().item()
            false_negative += torch.logical_and(y_pred != y, y == 1).sum().item() 
            false_positive += torch.logical_and(y_pred != y, y == 0).sum().item()
        

    recall = true_positive/(true_positive + false_negative)
    precision = true_positive/(true_positive + false_positive)
    specificity = true_negative/(false_positive + true_negative)
    f_score = (2*precision*recall)/(precision + recall)
    g_mean = (recall*specificity)**(0.5)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"\n recall: {recall} \n precision: {precision} \n specificity: {specificity} \n f_score: {f_score} \n g_mean: {g_mean}\n")

In [ ]:
model = Model().to(device)

In [ ]:
model

NameError: ignored

In [ ]:
loss_fn = nn.CrossEntropyLoss(weight = torch.Tensor([1.0, 1.0 ])).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
epochs = 1
for t in range(epochs):
    train(dataloader_train, model, loss_fn, optimizer)
    #test(dataloader_cross, model, loss_fn)
print("Done!")

loss: 0.673794  [    0/808615]
loss: 0.696484  [ 1500/808615]
loss: 0.701440  [ 3000/808615]
loss: 0.686184  [ 4500/808615]
loss: 0.669682  [ 6000/808615]
loss: 0.569941  [ 7500/808615]
loss: 0.631500  [ 9000/808615]
loss: 0.435020  [10500/808615]
loss: 0.639515  [12000/808615]
loss: 0.590054  [13500/808615]
loss: 0.739812  [15000/808615]
loss: 0.498859  [16500/808615]
loss: 0.610222  [18000/808615]
loss: 0.664825  [19500/808615]
loss: 0.751498  [21000/808615]
loss: 0.694819  [22500/808615]
loss: 0.554073  [24000/808615]
loss: 0.517511  [25500/808615]
loss: 0.580627  [27000/808615]
loss: 0.676046  [28500/808615]
loss: 0.601258  [30000/808615]
loss: 0.688710  [31500/808615]
loss: 0.573615  [33000/808615]
loss: 0.561130  [34500/808615]
loss: 0.581720  [36000/808615]
loss: 0.552546  [37500/808615]
loss: 0.502820  [39000/808615]
loss: 0.568191  [40500/808615]
loss: 0.527172  [42000/808615]
loss: 0.538314  [43500/808615]
loss: 0.513666  [45000/808615]
loss: 0.538747  [46500/808615]
loss: 0.

In [ ]:
X_train.type()

NameError: ignored